<a href="https://colab.research.google.com/github/otluiz/Covid-19_kaggle/blob/master/Copy_of_Covid_19_Kaggle_Textos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import zipfile
import pandas as pd
import glob
import json
import seaborn as sns
import spacy
import nltk
from IPython.core.display import HTML
from matplotlib import pyplot as plt

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
#!pip install -U spacy
#!python -m spacy download en
!pip install scispacy
import scispacy
!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.2.5/en_core_sci_md-0.2.5.tar.gz
import en_core_sci_md
import zipfile

Chama os drives da Google e monta um novo diretório neste 'notebook'. Precisa de uma conexão (URL) onde a Google gera uma 'auth' para entrar no meu Google Drive

First, you'll need to mount your Drive.

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Conecta ao diretório do meu Google Drive onde estão os dados da Covid-19 que está compactado.
Será descompactado (unzip) no meu Google Drive


In [ ]:
path = '/content/gdrive/My Drive/CORD-19-research-challenge.zip'
from zipfile import *
zip_object = zipfile.ZipFile(file=path, mode='r')
zip_object.extractall('./')
zip_object.close()

Criação do DataFrame com os textos dos artigos


DataFrame no formato JSON

In [48]:
features_corona = {'paper_id': [],
                   'title' : [],
                   'abstract' : [],
                   'text' : []}

Testando o tipo "features_corona"


In [10]:
type (features_corona)

dict

Transformar o tipo dict em dataframe


In [49]:
corona_df = pd.DataFrame.from_dict(features_corona)

Verificando o novo tipo

In [50]:
type (corona_df)

pandas.core.frame.DataFrame

Pandas é o "default for Machine Learning in Python projects"


In [ ]:
corona_df.head()

Ler os arquivos JSON a partir da raiz '/' para popular o futuro dataframe (corona_df) a ser criado mais adiante


In [ ]:
json_filenames = glob.glob(f'{"./"}//**/*.json', recursive = True)
print(json_filenames)

In [53]:
json_filenames)

29316

Cada arquivo JSON seráá uma linha do dataframe, populando assim o corona_df


In [54]:
def retorna_corona_df(json_filenames, df):
  for file_name in json_filenames:
    linha_dados = {'paper_id': None, 'title': None, 'abstract': None, 'text': None}

    with open(file_name) as json_data:
      if file_name == './sample_data/anscombe.json': # ignora os anscombe.json
        continue
      #if file_name == './MeuMestrado/ProjetoGeolocalização/node_modules/gulp/node_modules/tildify/*.json': 
      #  continue

      data = json.load(json_data)

      ## .strip() : ' 345 ' -> '345' retira os espaços
      linha_dados['paper_id'] = data['paper_id'].strip()
      linha_dados['title'] = data['metadata']['title'].strip()

      abstract_list = [abstract['text'] for abstract in data['abstract']]
      abstract = '\n'.join(abstract_list)
      linha_dados['abstract'] = abstract.strip()

      text_list = [text['text'] for text in data['body_text']]
      text = '\n'.join(text_list)
      linha_dados['text'] = text.strip()

      df = df.append(linha_dados, ignore_index = True)
  return df

In [55]:
corona_df = retorna_corona_df(json_filenames, corona_df)

In [56]:
corona_df2 = corona_df[['paper_id', 'title', 'abstract', 'text']] ## criando um back-up do DataFrame


In [19]:
#corona_df.drop(corona_df.tail(29315).index,inplace=True) # remove as ultimas linhas

In [20]:
#corona_df.drop(corona_df.head(2).index,inplace=True) # drop first 2 rows

In [57]:
corona_df.shape

(29315, 4)

In [22]:
corona_df2.shape

(29315, 4)

In [ ]:
corona_df.tail() ## Mostra os últimas 5 linhas do Dataframe corona_df

In [ ]:
## Verificando a coluna tíítulo (title) do último registro (29314) do Dataframe
corona_df['title'][29314] 


In [ ]:
corona_df['text'][2]  ## Verificando a coluna 'text' do Dataframe

In [64]:
len(corona_df['text'][2]) ## Quantas palavras têm este texto [2]?

36685

**PERPROCESSAMENTO DE TEXTOS DOS ARTIGOS**
<PROCURANDO POR DADOS FALTANTES>

In [ ]:
sns.heatmap(corona_df.isnull()); ## procurando dados faltantes pelo mapa de calor

In [66]:
len(corona_df[corona_df['paper_id'] == '']) # procurando por espaços vazios

0

In [67]:
len(corona_df[corona_df['title'] == '']) # procurando por espaços vazios nos tíítulos

3583

In [68]:
len(corona_df[corona_df['abstract'] == '']) # procurando por espaços vazios nos abstracts

8051

In [32]:
len(corona_df[corona_df['text'] == '']) # procurando por espaços vazios nos textos

0

In [69]:
corona_df = corona_df[corona_df['title'] != ''] ## apagando os artigos sem tíítulos

In [72]:
corona_df = corona_df[corona_df['abstract'] != ''] ## apagando os artigos sem abstract

**REMOVENDO ARTIGOS DUPLICADOS**

In [76]:
corona_df.drop_duplicates(['abstract', 'text', 'title'], inplace=True)

In [77]:
corona_df_bck = corona_df  ## reafzendo tudo porra!!

In [78]:
corona_df.shape ## o que sobrou após a limpeza??

(20211, 4)

In [79]:
corona_df_bck.shape  ## vendo quem mandou 2 ou mais vezes o mesmo artigo - ahaha

(20211, 4)

**BASE DE DADOS DE AMOSTRA**


In [80]:
corona_df = corona_df.sample(n = 500, random_state=1)

In [81]:
corona_df.shape

(500, 4)

In [ ]:
corona_df.head()

In [ ]:
corona_df.tail()

In [ ]:
## Visual Text analisys
sample_text = corona_df['text'][23192]
sample_text

**FUNÇÃO PARA PREPROCESSAMENTO**


In [85]:
#nlp = spacy.load('en')
#nlp = spacy.load('en_core_sci_md')
nlp = en_core_sci_md.load(disable=['tagger', 'parser', 'ner'])

nlp.max_length = 2000000

In [ ]:
#print('scispacy:', scispacy.__version__, " - ", "spacy:", spacy.__version__)
#!stopwnltk = nltk.corpus.stopwords.words("english")

In [ ]:
print(spacy.lang.en.stop_words.STOP_WORDS)

In [90]:
len(spacy.lang.en.stop_words.STOP_WORDS)

326

In [144]:
def spacy_tokenizer(sentence):
  sentence = sentence.lower() ## coloca tudo em minúsculo
  list = [] 
  list = [word.lemma_ for word in nlp(sentence) if not (word.is_stop or ## disable=['tagger', 'parser', 'ner']
                                                        word.like_num or
                                                        word.is_punct or
                                                        word.is_space)] 
  lsit = ' ' .join([str(element) for element in list])

  return list

In [ ]:
test = sample_text
result = spacy_tokenizer(test)
result

In [142]:
token_list = []
## junta as palavras numa mesma linha
def join_sentences(textos):
  token_list = ' '.join([token_list for token_list in textos])
  return token_list

In [ ]:
sample_text

In [ ]:
test = '1 ' + sample_text
result = join_sentences(spacy_tokenizer(test))
result

In [184]:
corona_df['text'] = corona_df['text'].apply(join_sentences) #  spacy_tokenizer

In [ ]:
print(sample_text)

In [185]:
len(sample_text)

15149

In [ ]:
print(corona_df['text'][23192])

In [151]:
len(corona_df['text'][23192])

1271

In [ ]:
for index, row in corona_df.iterrows():
  print(row['paper_id'], row['title']) # index mostra o índice de cada artigo
  text_file = open('corpus/' + row['paper_id'] + '.txt', 'w')
  n = text_file.write(str(row['text']))
  text_file.close()

A função PlaintextCorpusReader vai ler as frases como corpus
de todos os (agora) txt que estão na novo diretório "corpus/"


In [187]:
from nltk.corpus import PlaintextCorpusReader
corpus = PlaintextCorpusReader('corpus', '.*')

In [ ]:
files = corpus.fileids() # visualizer se os aruivos foram criados
files[0]

In [ ]:
corpus.raw('00326efcca0852dc6e39dc6b7786267e1bc4f194.txt')

In [ ]:
words = corpus.words()
print(words)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [173]:
len(words)

3739523

In [ ]:
frequenc = nltk.FreqDist(words)
most_common = frequenc.most_common(100) # palavras mais comuns
most_common

**NUVEM DE PALAVRAS**

In [175]:
from matplotlib.colors import ListedColormap
color_map = ListedColormap(['orange', 'green', 'red', 'magents'])

In [177]:
from wordcloud import WordCloud
cloud = WordCloud(background_color = 'white', max_words = 100, colormap = color_map)

In [ ]:
cloud = cloud.generate(corona_df['text'].str.cat(sep = '\n'))
plt.figure(figsize=(15,15))
plt.axix('off')
plt.imshow(cloud)
plt.show()